### 使用ctype模块调用动态链接库中导出的函数
#### 动态链接库函数的解析与访问
1. 动态链接库本身是一些经过编译的二进制文件，只在运行时才会被链接进主进程。
    + Windows平台这些二进制文件被称为动态链接库（DLL，Dynamic Link Library）
    + Linux平台则为共享对象（SO，Shared Object）
    + macOS平台则为dylib
2. 这些二进制文件通过导出函数名称的方式来呈现它们所包含的函数，而函数名称会被解析成内存中实际的函数地址。
3. 在Python中调用这些链接库函数需要自行解析出这些导出函数的地址，这就是ctype模块的作用。

#### ctype模块提供的三种动态链接库加载方式
1. `cdll()`加载遵循cdecl标准函数调用约定的链接库。
2. `windll()`加载遵循stdcall调用约定的动态链接库，stdcall是微软**Win32 API**使用的原生调用约定。
3. `oledll()`与`windll()`类似，但`oledll()`会假定其载入的函数会统一返回一个Windows HRESULT错误编码，该编码专门服务于微软的COM（组件对象模型）函数，用于表示错误信息。

In [6]:
#!/usr/bin python3
# chapter1-printf.py
from ctypes import *
msvcrt = cdll.msvcrt # Windows
# libc = CDLL("libc.so") # Linux
# libc = CDLL("libc.dylib") # macOS
msg_str = "hello world!\n"
msvcrt.wprintf("Testing: %s", msg_str)
# msg_str = b"hello world!\n"
# msvcrt.printf(b"Testing: %s", msg_str)

22

Python交互式shell等会输出字符串的长度（算上'\0'，在此为22）
```jupyter
Out: 22
```
而测试字符串是在终端输出！！
```bash
Testing: hello world!
```

构建C数据类型，下表是基本数据类型在C、Python以及ctypes类型之间的转换和对应关系。

| C 类型 | Python 类型 | ctypes 类型 |
| :--- | :--- | :--- |
| `_Bool` | bool (1) | c_bool |
| `char` | 1-character **bytes** object | c_char |
| `wchar_t` | 1-character string <br> (in Python3 string is **unicode**) | c_wchar |
| `char` | int | c_byte |
| `unsigned char` | int | c_ubyte |
| `short` | int | c_short |
| `unsigned short` | int | c_ushort |
| `int` | int | c_int |
| `unsigned int` | int | c_uint |
| `long` | int | c_long |
| `unsigned long` | int | c_ulong |
| `__int64` or `long long` | int | c_longlong |
| `unsigned __int64` <br> or `unsigned long long` | int | c_ulonglong |
| `size_t` | int | c_size_t |
| `float` | float | c_float |
| `double` | float | c_double |
| `long double` | float | c_longdouble |
| `char *` (NUL terminated) | bytes object or None | c_char_p |
| `wchar_t *` (NUL terminated) | string or None | c_wchar_p |
| `void *` | int or None | c_void_p |

下面是基本类型示例代码和如何用ctype模块定义结构体和联合体。

In [7]:
# 示例代码，最好在Python交互式Shell输入，可以免去print()函数。
from ctypes import *
print(c_int())
print(c_ushort(-5))
print(c_char_p(b"Hello world"))
a = c_wchar_p("Hello world")
print(a.value)

c_long(0)
c_ushort(65531)
c_char_p(1667176826064)
Hello world


In [3]:
# 定义结构体
from ctypes import *
'''
struct A{
    int a;
    int b;
}
'''
class A(Structure):
    _fields_ = [
        ("a", c_int),
        ("b", c_int),
    ]

# 定义联合体
'''
union{
    long b_l;
    int b_i;
    char b_c[8];
}B
'''
class B(Union):
    _fields_ = [
        ("b_l", c_long),
        ("b_i", c_int),
        ("b_c", c_char * 8),
    ]

my_B = B(66)
print("B as a long: %ld" % my_B.b_l)
print("B as a int: %d" % my_B.b_i)
print("B as a char: %s" % my_B.b_c)

B as a long: 66
B as a int: 66
B as a char: b'B'


### 函数调用的约定
1. 函数调用约定描述了如何以正确的方式调用某些特定类型的函数。包括：
    + 函数参数在栈上的分配顺序
    + 哪些参数会被压入栈中
    + 哪些参数将通过寄存器传入
    + 函数返回时函数栈的回收方式
2. 两种最基本的函数调用约定：
    + `cdecl`：规定函数的参数列表以从右向左的顺序入栈，并有函数的调用者负责清除栈上的参数。大多数
    + `stdcall`